In [207]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta, time
from bs4 import BeautifulSoup
import requests
import numpy as np
from functools import reduce
import re
from unidecode import unidecode
import time

# sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# pandas data reader
import pandas_datareader.data as web
from datetime import datetime as dt
from scipy.stats import pearsonr

# Alpha Vantage requirements
av_api_key = 'TGXS2LEUKMHXTKT5'
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries( key = av_api_key, output_format = 'pandas')

# plotting
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
# Required for Plot.ly Offline
init_notebook_mode(connected=True)
# Cufflinks bind plotly to pandas dataframe in IPython Notebooks
cf.set_config_file(offline = False, world_readable = True, theme = 'ggplot')

In [275]:
l_symbols = ['FB','AMZN', 'NFLX', 'GOOGL', 'QQQ', 'SPY']
sdate = dt(2018,9,1)
edate = dt(2019,2,1)
yhoo_data = web.DataReader( l_symbols, 'yahoo', sdate, edate)

In [317]:
yhoo_data.head(5)

Attributes         High                                                   \
Symbols            AMZN          FB        GOOGL        NFLX         QQQ   
Date                                                                       
2018-08-31  2022.380005  177.619995  1252.550049  376.000000  187.179993   
2018-09-04  2050.500000  173.889999  1227.869995  368.880005  186.399994   
2018-09-05  2040.380005  171.130005  1214.869995  363.390015  185.550003   
2018-09-06  2007.500000  166.979996  1198.989990  356.000000  183.750000   
2018-09-07  1975.199951  164.630005  1188.550049  355.750000  182.669998   

Attributes                      Low                                       \
Symbols            SPY         AMZN          FB        GOOGL        NFLX   
Date                                                                       
2018-08-31  290.809998  2004.739990  174.979996  1225.750000  367.079987   
2018-09-04  290.209991  2013.000000  168.800003  1206.560059  361.260010   
2018-09-05  289.640015  1989.890015  166.669998  1175.339966  335.829987   
2018-09-06  289.489990  1935.209961  160.000000  1163.609985  341.989990   
2018-09-07  288.700012  1937.349976  160.160004  1169.270020  341.250000   

Attributes     ...         Volume                                      \
Symbols        ...          GOOGL        NFLX         QQQ         SPY   
Date           ...                                                      
2018-08-31     ...      2115400.0   7943400.0  28653200.0  66140800.0   
2018-09-04     ...      2072900.0   7605200.0  29063500.0  57594400.0   
2018-09-05     ...      2699400.0  13092800.0  42623300.0  72452400.0   
2018-09-06     ...      2342700.0  13050200.0  46091400.0  65909900.0   
2018-09-07     ...      1632900.0   9105800.0  46629500.0  73524800.0   

Attributes    Adj Close                                                   \
Symbols            AMZN          FB        GOOGL        NFLX         QQQ   
Date                                                                       
2018-08-31  2012.709961  175.729996  1231.800049  367.679993  185.783188   
2018-09-04  2039.510010  171.160004  1211.310059  363.600006  184.986923   
2018-09-05  1994.819946  167.179993  1199.099976  341.179993  182.598038   
2018-09-06  1958.310059  162.529999  1183.989990  346.459991  180.965668   
2018-09-07  1952.069946  163.039993  1177.589966  348.679993  180.268921   

Attributes              
Symbols            SPY  
Date                    
2018-08-31  287.323883  
2018-09-04  286.829010  
2018-09-05  286.057037  
2018-09-06  285.196014  
2018-09-07  284.641754  

[5 rows x 36 columns]

In [318]:
data = yhoo_data.stack()
data.reset_index(inplace = True)

l_col_to_keep = ['Date', 'Symbols', 'Open', 'Adj Close', 'Volume']
r_dfs = []

# 1
for sym in l_symbols:
    r_df = data[ data.Symbols == sym ].loc[:, l_col_to_keep]
    r_df = r_df.set_index('Date')
    r_df[f'r({sym})'] = r_df['Open']/ r_df['Adj Close'].shift(1) - 1
    # or just do df.pct_change()
    r_dfs.append( r_df.iloc[1:,:])

# 2
return_ = reduce( lambda x, y : pd.concat([x,y], axis =1),
            [ r_df.iloc[:,-1] for r_df in r_dfs ]
           )

return_.sort_values(by = 'Date',ascending = False).head(3)

,r(FB),r(AMZN),r(NFLX),r(GOOGL),r(QQQ),r(SPY)
Date,,,,,,
2019-02-01,-0.005099,-0.046459,-0.006834,-0.003197,-0.004698,0.000815
2019-01-31,0.100917,0.013422,-0.002877,0.012978,0.006156,-0.000262
2019-01-30,0.014079,0.018270,0.011706,0.006822,0.011326,0.006416


In [279]:
from scipy import stats
def GetBeta( r_sym , r_benchmark):
    slope, intercept, r_value, p_value, std_err = stats.linregress( r_sym, r_benchmark)
    return slope

def GetAlpha( r_sym , r_benchmark):
    beta = GetBeta( r_sym, r_benchmark)
    return r_sym - beta * r_benchmark

In [218]:
def scrape_nasdaq(stockcode, last_page_num):
    """
    Stockcode: FB/NFLX/AMZN/GOOGL
    Last_page: The last page of the latest news page of a specific stock
    """
    page_num = 1
    results  = []
    while True:
        if page_num > last_page_num: # Last page
            break
        html = requests.get(f'https://www.nasdaq.com/symbol/{stockcode}/news-headlines?page={page_num}')
        soup = BeautifulSoup(html.text, "html.parser")
        for divs in soup.find_all(class_ = 'news-headlines'):
            headline = [s.text for s in divs.find_all('a', {'target':"_self"})]
            datetime = [d.text.strip() for d in divs.find_all('small')]
            for i in range(len(headline)):
                dict_ = {'headline': headline[i], 'datetime': datetime[i]}
                results.append(dict_)
        page_num += 1
    return pd.DataFrame(results)

In [147]:
def combine_csv():
    """
    This function merges the scraped data in csv format into a single csv file
    """
    list_ = []
    for f in glob.glob('/dataset/*.csv'):
        df = pd.read_csv(f, index_col=None, header=0)
        list_.append(df)
        frame = pd.concat(list_, axis = 0, ignore_index = True)
        del frame['Unnamed: 0']
    frame.datetime = pd.to_datetime(frame.datetime)
    return frame

In [284]:
sent_df = pd.read_csv('merged.csv')
del sent_df['Unnamed: 0']

In [285]:
# change the timezone from UTC to EST
sent_df.datetime = pd.to_datetime(sent_df.datetime).apply(lambda x: x - timedelta(hours=5))

In [286]:
def time_tuner(date):
    """
    This function:
    (1) distinguishes summer hours from winter hours.
    (2) distinguishes open trading hours from close trading hours.
    (3) assigns 'NA' to headlines published within trading hours.
    (4) counts headlines posted on weekdays BEFORE trading hours towards that day.
    (5) counts headlines posted on weekdays AFTER trading hours towards the next day,
        except Friday.
    (6) counts headlines posted on Friday AFTER trading hours towards the next Monday.
    (7) counts headlines posted on Saturday and Sunday towards the next Monday.  
    """
    
    # define the summer hours in 2018 of the New York Stock Exchange
    summer_time_start = dt.strptime('2018-03-11 01:59:59', "%Y-%m-%d %H:%M:%S")
    summer_time_end   = dt.strptime('2018-11-04 02:00:01', "%Y-%m-%d %H:%M:%S")
    
    # define the trading hours in both summer time and winter time
    sum_open_trade    = dt.strptime('09:29:59', "%H:%M:%S").time()
    sum_close_trade   = dt.strptime('16:00:01', "%H:%M:%S").time()
    win_open_trade    = dt.strptime('10:29:59', "%H:%M:%S").time()
    win_close_trade   = dt.strptime('17:00:01', "%H:%M:%S").time()
    
    # If the tweets were posted within the SUMMER time
    if (date > summer_time_start) and (date < summer_time_end):
        
        if date.weekday() == 6: #Sunday 
            date = date + timedelta(days=1) 
            date = date.strftime('%Y-%m-%d')
        
        elif date.weekday() == 5: #Saturday
            date = date + timedelta(days=2)
            date = date.strftime('%Y-%m-%d')
        
        elif date.weekday() == 4: # Friday
    
            if ((date.time() > sum_open_trade) and (date.time() < sum_close_trade)):
                date = 'NA'
            
            elif (date.time() < sum_close_trade):
                date = date.strftime('%Y-%m-%d')
            
            else:
                
                date = date + timedelta(days=3)
                date = date.strftime('%Y-%m-%d')
        else: 
            
            if ((date.time() > sum_open_trade) and (date.time() < sum_close_trade)):
                date = 'NA'
            
            elif (date.time() < sum_close_trade):
                date = date.strftime('%Y-%m-%d')
            
            else:
                date = date + timedelta(days=1)
                date = date.strftime('%Y-%m-%d')
    
    else:
        
        if date.weekday() == 6: #Sunday 
            date = date + timedelta(days=1) 
            date = date.strftime('%Y-%m-%d')
            
        elif date.weekday() == 5: #Saturday
            date = date + timedelta(days=2)
            date = date.strftime('%Y-%m-%d')
            
        elif date.weekday() == 4: # Friday
            # before and within trading hours
            if ((date.time() > win_open_trade) and (date.time() < win_close_trade)):
                date = 'NA'
                
            elif (date.time() < win_close_trade):
                date = date.strftime('%Y-%m-%d')
            else:
                # after trading hours
                date = date + timedelta(days=3)
                date = date.strftime('%Y-%m-%d')
        else: 
            
            if ((date.time() > win_open_trade) and (date.time() < win_close_trade)):
                date = 'NA'
            
            elif (date.time() < sum_close_trade):
                date = date.strftime('%Y-%m-%d')
            
            else:
                date = date + timedelta(days=1)
                date = date.strftime('%Y-%m-%d')              
    
    return date

In [287]:
len(sent_df.datetime) # total rows 

5496

In [288]:
# rows left after filtering out the headlines within trading hours
len(sent_df[sent_df.datetime.apply(time_tuner) != 'NA']) 

3788

In [289]:
len(sent_df[sent_df.datetime.apply(time_tuner) != 'NA']) / len(sent_df.datetime)

0.6892285298398836

In [290]:
sent_df['Date'] = sent_df.datetime.apply(time_tuner)

In [291]:
sent_df = sent_df[sent_df['Date'] != 'NA']
sent_df.set_index('Date', inplace=True)

In [292]:
sent_df['datetime_weekday'] = sent_df['datetime'].apply(lambda x: x.weekday())

In [293]:
analyzer = SentimentIntensityAnalyzer()
sent_df['vader'] = sent_df['headline'].apply(lambda x: analyzer.polarity_scores(unidecode(x))['compound'])


In [294]:
sent_df[['headline','vader']]['headline'][7]

'Qualcomm to Add Amazon Music, Video to Your Driving Experience'

In [295]:
sent_df.tail(5)

,datetime,headline,stockcode,datetime_weekday,vader
Date,,,,,
2018-11-06,2018-11-06 10:03:00,US STOCKS-Wall St holds small gain as U.S. vot...,NFLX,1,0.2624
2018-11-05,2018-11-05 07:21:00,"Oil stocks, Berkshire lift S&P 500; Apple push...",NFLX,0,0.0261
2018-11-05,2018-11-05 07:21:00,"US STOCKS-Oil stocks, Berkshire lift S&P 500; ...",NFLX,0,0.0982
2018-11-05,2018-11-05 05:16:00,"Energy stocks lift S&P, Dow; Apple drags",NFLX,0,0.4557
2018-11-05,2018-11-05 05:16:00,"US STOCKS-Energy stocks lift S&P, Dow; Apple d...",NFLX,0,0.2728


In [328]:
# getting alpha for each stock
alpha_fb   = GetAlpha( df['r(FB)'], df['r(QQQ)'])
alpha_amzn = GetAlpha( df['r(AMZN)'], df['r(QQQ)'])
alpha_nflx = GetAlpha( df['r(NFLX)'], df['r(QQQ)'])
alpha_googl = GetAlpha( df['r(GOOGL)'], df['r(QQQ)'])

# getting sentiment for each stock
NFLX = sent_df[sent_df['stockcode'] == 'NFLX']
FB   = sent_df[sent_df['stockcode'] == 'FB']
AMZN = sent_df[sent_df['stockcode'] == 'AMZN']
GOOGL = sent_df[sent_df['stockcode'] == 'GOOGL']

def daily_sentiment_calculator(data):
    data = data.reset_index()
    data = data.groupby(data['Date'])['vader'].sum()
    return data

def get_alpha_sent_table(alpha, data):
    """
    alpha: the alpha of the stock
    data : the sentiment data of the stock
    stockcode: just for you to input a stock code
    """
    data = daily_sentiment_calculator(data)
    data = data.reset_index()
    data.columns = ['Date', 'Vader']
    data['Date'] = pd.to_datetime(data['Date'])
    alpha = alpha.reset_index()
    alpha.columns = ['Date','Alpha']
    merge = pd.merge(data, alpha, how='inner', on='Date')
    return merge

def alpha_sent_stat_analysis(table):
    """
    calculate pearson r correlation cofficient and its p-value
    null hypothesis: no correlation between two variables
    alternative hypothesis: they are correlated
    threshold: if p < 0.05, the null hypothesis should be rejected
    """
    Vader = pearsonr(table['Vader'], table['Alpha'])
    return {'Vader_corr':round(Vader[0],2), 'Vader_p_value':round(Vader[1],2)}

def plot_alpha_corr_vader(table):
    scatterdf = table.dropna()[['Alpha','Vader']]
    plotly_data = [go.Scatter(x = scatterdf.Alpha, y = scatterdf.Vader, 
                              mode = 'markers',
                              marker = dict(size = 10,  
                                            line = dict( width = 2,)
                                           )
                             )]
    layout = go.Layout(title = 'excess return vs sentiments',
                       xaxis = {'title': 'excess return'},
                       yaxis = {'title': 'vader sentiment'})
    fig = go.Figure(data = plotly_data, layout= layout)
    return iplot(fig)

In [310]:
NFLX_table = get_alpha_sent_table(alpha_nflx, NFLX)
FB_table = get_alpha_sent_table(alpha_fb, FB)
AMZN_table = get_alpha_sent_table(alpha_amzn, AMZN)
GOOGL_table = get_alpha_sent_table(alpha_googl, GOOGL)

In [313]:
print('NFLX')
print(alpha_sent_stat_analysis(NFLX_table))
plot_alpha_corr_vader(NFLX_table)

NFLX
{'Vader_corr': 0.18, 'Vader_p_value': 0.1}


In [314]:
print('FB')
print(alpha_sent_stat_analysis(FB_table))
plot_alpha_corr_vader(FB_table)

FB
{'Vader_corr': 0.15, 'Vader_p_value': 0.18}


In [315]:
print('GOOGL')
print(alpha_sent_stat_analysis(GOOGL_table))
plot_alpha_corr_vader(GOOGL_table)

GOOGL
{'Vader_corr': 0.03, 'Vader_p_value': 0.8}


In [316]:
print('AMZN')
print(alpha_sent_stat_analysis(AMZN_table))
plot_alpha_corr_vader(AMZN_table)

AMZN
{'Vader_corr': 0.54, 'Vader_p_value': 0.0}


After looking into the correlation between Alpha and sentiment on a daily basis, we're also interested in the stock return between the open price and the close price of the previous day. 

In [322]:
# Getting return between today's open price and ytd's adj close price
return_FB    = return_['r(FB)']
return_AMZN  = return_['r(AMZN)']
return_GOOGL = return_['r(GOOGL)']
return_NFLX  = return_['r(NFLX)']

In [323]:
r_NFLX_table = get_alpha_sent_table(return_NFLX, NFLX)
r_FB_table = get_alpha_sent_table(return_FB, FB)
r_AMZN_table = get_alpha_sent_table(return_AMZN, AMZN)
r_GOOGL_table = get_alpha_sent_table(return_GOOGL, GOOGL)

In [324]:
print('NFLX')
print(alpha_sent_stat_analysis(r_NFLX_table))
plot_alpha_corr_vader(r_NFLX_table)

NFLX
{'Vader_corr': 0.18, 'Vader_p_value': 0.09}


In [325]:
print('FB')
print(alpha_sent_stat_analysis(r_FB_table))
plot_alpha_corr_vader(r_FB_table)

FB
{'Vader_corr': 0.17, 'Vader_p_value': 0.13}


In [326]:
print('AMZN')
print(alpha_sent_stat_analysis(r_AMZN_table))
plot_alpha_corr_vader(r_AMZN_table)

AMZN
{'Vader_corr': 0.52, 'Vader_p_value': 0.0}


In [327]:
print('GOOGL')
print(alpha_sent_stat_analysis(r_GOOGL_table))
plot_alpha_corr_vader(r_GOOGL_table)

GOOGL
{'Vader_corr': 0.11, 'Vader_p_value': 0.32}
